In [ ]:
# !pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests


import time
import random

In [ ]:
driver = webdriver.Chrome()
# Islamabad Page
driver.get('https://www.zameen.com/new-projects/islamabad-3-1/')


# Get all the anchor tags
all_projects_selector = 'main > div > div > div > div > div > a'

elements = driver.find_elements(By.CSS_SELECTOR, all_projects_selector)


In [ ]:
type(driver)

##### Loop through Tags and extract the links

In [ ]:
def html_to_file(html, output_file = 'output.html'):
    pretty_soup = BeautifulSoup(html, 'html.parser').prettify()
    # Write the BeautifulSoup object to an HTML file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(pretty_soup)

1. Get Links for all the pages projects in the City
    TO Do this, Open the city page, get all the links,
    Do this till gone through all the pages of a city.
2. Then open these links in selenium one by page, and gather information for each project

In [ ]:
url = f'https://www.zameen.com/new-projects/islamabad-3-1/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

offering_divs = soup.select("main > div > div > div > div > div > a")

offering_divs


In [ ]:
urls = [ele.get('href') for ele in offering_divs]

In [ ]:
class City:
    def __init__(self, url) -> None:
        self.url = url
    
    def _get_total_pages(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.text)

        total_pages = int(soup.select_one('ul[aria-label="Pagination"] > li:nth-last-child(2) > a').text)

        return total_pages

    def get_all_project_links(self):
        
        city_project_urls = []
        total_pages = self._get_total_pages()

        for page in range(1, total_pages+1):
            time.sleep(random.randint(1, 5))
            try:
                response = requests.get(f'https://www.zameen.com/new-projects/islamabad-3-1/?page={page}')
                    
                soup = BeautifulSoup(response.text)
                proj_tags = soup.select("main > div > div > div > div > div > a")
                urls = [ele.get('href') for ele in proj_tags]
                city_project_urls.extend(urls)
            
            except requests.exceptions.ReadTimeout:
                time.sleep(10)
                continue
            
        return city_project_urls
    
    def scrap(self):
        pass
        

In [ ]:
city_soup = BeautifulSoup(requests.get('https://www.zameen.com/new-projects/islamabad-3-1/').text)

In [ ]:
# There is only a single ul having area-label of Pagination
total_pages = int(city_soup.select_one('ul[aria-label="Pagination"] > li:nth-last-child(2) > a').text)
total_pages

In [ ]:
range(1, total_pages+1)

In [ ]:
city_project_urls = []
for page in range(1, total_pages+1):
    time.sleep(random.randint(1, 5))
    try:
        response = requests.get(f'https://www.zameen.com/new-projects/islamabad-3-1/?page={page}')
            
        soup = BeautifulSoup(response.text)
        proj_tags = soup.select("main > div > div > div > div > div > a")
        urls = [ele.get('href') for ele in proj_tags]
        city_project_urls.extend(urls)
    
    except requests.exceptions.ReadTimeout:
        time.sleep(10)
        continue

In [ ]:
try:
    pass

except requests.exceptions.ReadTimeout:
    pass

In [ ]:
city_project_urls

In [2]:
def get_text_javascript(driver, ele):
    """Get text of the element using javascript
    This allows getting text even for hidden elements
    """
    return driver.execute_script("return arguments[0].textContent;", ele)

In [8]:
class Page:
    def __init__(self, url) -> None:
        self.url = url

    def scrap(self, driver: WebDriver):
        driver.get(self.url)


        # Wait for the page to be fully loaded
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )


        project_name_selector = 'main > div > div > div:nth-child(1) > div > div:nth-child(2) > h1'
        project_location_selector = 'main > div> div > div:nth-child(1) > div > div:nth-child(2) > div > div > div'

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, project_name_selector))
            )

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, project_location_selector))
            )

        project_name = get_text_javascript(driver.find_element(By.CSS_SELECTOR, project_name_selector))
        project_location = get_text_javascript(driver.find_element(By.CSS_SELECTOR, project_location_selector))

        project_data = self._get_data(driver)
        
        # Create new dict with some info and then update it with the project_data dict
        # This way, the added information will appear at top of the dict
        data_dict = {'project_name': project_name, 'project_location': project_location}
        data_dict.update(project_data)



        return data_dict

    
    def _get_overview(self, driver):
        overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

        offerings = overview_elements[0].text
        offerings = list(map(lambda x: x.strip(), offerings.split(' ')))


        developer = overview_elements[1].text

    def _get_data(self, driver):
    
        this_project_data = {}

        overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

        offerings = overview_elements[0].text
        offerings = list(map(lambda x: x.strip(), offerings.split(' ')))


        developer = overview_elements[1].text

        this_project_data['offerings'] = offerings
        this_project_data['developer'] = developer

        info_parent_element = driver.find_element(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(2) > div > div')
        info_elements = info_parent_element.find_elements(By.CSS_SELECTOR, ':scope > div')

        tile_data = []
        for info_ele in info_elements:
            info_ele_data = {}
            # Get type name
            
            # Open the element and trigger it's loading
            info_ele.click()

            
            time.sleep(random.random()) # Some sleep to ensure loading 

            type_name = get_text_javascript(info_ele.find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_headerTitle"]'))
            
            # Get price range
            price_range = get_text_javascript(info_ele.find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_price"]'))
            
            if type_name.casefold() == 'shops':
                continue
            #Get the info cards within this type

            info_ele_data['type'] = type_name
            info_ele_data['price_range'] = price_range

            info_cards = info_ele.find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
                    
            
            # Get info from each card
            cards_data_list = []
            for card in info_cards:
                card_data = {}


                # match type_name.casefold():
                #     case 'flats':
                #         (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')
            
                #     case _:
                #         (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')

                (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')
                card_type, card_price = get_text_javascript(card_type_ele), get_text_javascript(card_price_ele)

                card_data['type'] = card_type
                card_data['price'] = card_price

                # Info blocks within a card
                info_blocks = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')
                
                # A single key would not be repeated within a block
                for block in info_blocks:
                    #property name
                    property_name = get_text_javascript(block.find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]'))
                    #property value
                    property_val = get_text_javascript(block.find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]'))

                    card_data[property_name] = property_val

                cards_data_list.append(card_data)
            
            info_ele_data['cards'] = cards_data_list

            tile_data.append(info_ele_data)

        this_project_data['tiles'] = tile_data

        return this_project_data




    

In [ ]:
# html_to_file(city_soup.select_one('ul[aria-label="Pagination"]').prettify(), 'pagi.html')

In [ ]:
driver = webdriver.Chrome()

proj_base_url = 'https://www.zameen.com'
page_url = proj_base_url + city_project_urls[2]
driver.get(page_url)

# Wait for the page to be fully loaded
WebDriverWait(driver, 10).until(
    lambda driver: driver.execute_script("return document.readyState") == "complete"
)


In [ ]:
proj_base_url = 'https://www.zameen.com'
page_url = proj_base_url + city_project_urls[2]


In [9]:
driver = webdriver.Chrome()
# proj_base_url = 'https://www.zameen.com'
# page_url = proj_base_url + city_project_urls[2]

page_url = "https://www.zameen.com/new-projects/360_the_residences-2241/"


page = Page(page_url)
project_data = page.scrap(driver)

In [10]:
project_data

{'project_name': '360 The Residences',
 'project_location': 'Bhara kahu, Islamabad',
 'offerings': ['Flats,', 'Flats'],
 'developer': 'Hajvairy Developers',
 'tiles': [{'type': 'Flats',
   'price_range': 'PKR 1.53 Crore to 6.65 Crore',
   'cards': [{'type': '1 Bed Classic Apartment',
     'price': 'PKR 1.53 Crore',
     'Area Size ': '3.68 marla',
     'Bedrooms ': '1',
     'Bathrooms ': '1'},
    {'type': '1 BEDS FLATS (EXECUTIVE ON 1ST - 12TH FLOOR)',
     'price': 'PKR 1.56 Crore',
     'Area Size ': '3.76 marla',
     'Bedrooms ': '1',
     'Bathrooms ': '1'},
    {'type': '2 BEDS FLATS (EXECUTIVE ON 1ST - 12TH FLOOR)',
     'price': 'PKR 2.2 Crore',
     'Area Size ': '5.31 marla',
     'Bedrooms ': '2',
     'Bathrooms ': '2'},
    {'type': '3 BEDS FLATS (GOLD ON 1ST - 12TH FLOOR)',
     'price': 'PKR 3.31 Crore',
     'Area Size ': '7.99 marla',
     'Bedrooms ': '3',
     'Bathrooms ': '3'},
    {'type': '1 BEDS FLATS (PREMIUM ON 1ST - 12TH FLOOR)',
     'price': 'PKR 1.65 Cro

In [ ]:
get_text_javascript(driver.find_element(By.CSS_SELECTOR, 'main > div > div > div:nth-child(1) > div > div:nth-child(2) > h1'))
get_text_javascript(driver.find_element(By.CSS_SELECTOR, 'main > div> div > div:nth-child(1) > div > div:nth-child(2) > div > div > div'))

In [ ]:
# driver.find_elements(By.CSS_SELECTOR, 'h2[class*="heading"]')
overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

offerings = overview_elements[0].text
developer = overview_elements[1].text

In [ ]:
list(map(lambda x: x.strip(), offerings.split(' ')))

In [ ]:
info_parent_element = driver.find_element(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(2) > div > div')
info_parent_element.get_attribute('outerHTML')

In [ ]:
info_elements = info_parent_element.find_elements(By.CSS_SELECTOR, ':scope > div')
for ele in info_elements:
    ele.find_elements()
    # print(ele.get_attribute('outerHTML'))

In [ ]:
print(info_elements[0].find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_headerTitle"]').text)
print(info_elements[0].find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_price"]').text)

In [ ]:
class ElementData:
    
    @staticmethod
    def extract_flats_data():
        pass

In [ ]:
def get_text_javascript(ele):
    """Get text of the element using javascript
    This allows getting text even for hidden elements
    """
    return driver.execute_script("return arguments[0].textContent;", ele)

In [ ]:
this_project_data = {}

overview_elements = driver.find_elements(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(1) > div > div[class^="new-projects-overview"] [class^="new-projects-overview_value"]')

offerings = overview_elements[0].text
offerings = list(map(lambda x: x.strip(), offerings.split(' ')))


developer = overview_elements[1].text

this_project_data['offerings'] = offerings
this_project_data['developer'] = developer

info_parent_element = driver.find_element(By.CSS_SELECTOR, 'main > div > div:nth-child(2) > div > div:nth-child(2) > div > div')
info_elements = info_parent_element.find_elements(By.CSS_SELECTOR, ':scope > div')

tile_data = []
for info_ele in info_elements:
    info_ele_data = {}
    # Get type name
    
    # Open the element and trigger it's loading
    info_ele.click()

    
    time.sleep(random.random()) # Some sleep to ensure loading 

    type_name = get_text_javascript(info_ele.find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_headerTitle"]'))
    
    # Get price range
    price_range = get_text_javascript(info_ele.find_element(By.CSS_SELECTOR, 'div.rc-collapse-header [class*="property-type-collpase_price"]'))
    
    if type_name.casefold() == 'shops':
        continue
    #Get the info cards within this type

    info_ele_data['type'] = type_name
    info_ele_data['price_range'] = price_range

    info_cards = info_ele.find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
            
    
    # Get info from each card
    cards_data_list = []
    for card in info_cards:
        card_data = {}


        # match type_name.casefold():
        #     case 'flats':
        #         (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')
    
        #     case _:
        #         (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')

        (card_type_ele, card_price_ele) = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')
        card_type, card_price = get_text_javascript(card_type_ele), get_text_javascript(card_price_ele)

        card_data['type'] = card_type
        card_data['price'] = card_price

        # Info blocks within a card
        info_blocks = card.find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')
        
        # A single key would not be repeated within a block
        for block in info_blocks:
            #property name
            property_name = get_text_javascript(block.find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]'))
            #property value
            property_val = get_text_javascript(block.find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]'))

            card_data[property_name] = property_val

        cards_data_list.append(card_data)
    
    info_ele_data['cards'] = cards_data_list

    tile_data.append(info_ele_data)

this_project_data['tiles'] = tile_data


In [ ]:
this_project_data

#### Extract Flats Data

In [ ]:
info_cards = info_elements[1].find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
# for card in info_cards:
#     print(card.get_attribute('outerHTML'))

(card_type_ele, card_price_ele) = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_cardHeader"] > div > div')


print(driver.execute_script("return arguments[0].textContent;", card_type_ele), driver.execute_script("return arguments[0].textContent;", card_price_ele))

info_blocks = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')
#property name
name_ele = info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]')
#property value
price_ele = info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]')

print(driver.execute_script("return arguments[0].textContent;", name_ele))
print(driver.execute_script("return arguments[0].textContent;", price_ele))

#### Extract Shops Data

In [ ]:
info_cards = info_elements[1].find_elements(By.CSS_SELECTOR, 'div.rc-collapse-content-box div[class*="card_card"]')
# for card in info_cards:
#     print(card.get_attribute('outerHTML'))
# print(info_cards[0].get_attribute('outerHTML'))
# info_cards[0].find_elements(By.CSS_SELECTOR, 'div.flex.flexYcenter')

# (card_type_ele, card_price_ele) = info_cards[0].find_elements(By.CSS_SELECTOR, 'div.flex.flexYcenter')

# card_type_ele.get_attribute('outerHTML')
# print(get_text_javascript(card_type_ele))
# print(get_text_javascript(card_price_ele))

# print(info_cards[0].get_attribute('outerHTML'))
card_type_ele = info_cards[0].find_element(By.CSS_SELECTOR, ':scope > div > div:first-child > div > div:first-child')
card_price_ele = info_cards[0].find_element(By.CSS_SELECTOR, ':scope > div > div:nth-child(2)')
# card_property_ele = info_cards[0].find_element(By.CSS_SELECTOR, ':scope > div > div:first-child > div > div:nth-child(2)')
card_property_ele = info_cards[0].find_element(By.CSS_SELECTOR, ':scope > div > div:first-child > div > div:nth-child(2)')

get_text_javascript(card_price_ele)


# print(driver.execute_script("return arguments[0].textContent;", card_type_ele), driver.execute_script("return arguments[0].textContent;", card_price_ele))


# print(info_cards[0].get_attribute('outerHTML'))
# info_blocks = info_cards[0].find_elements(By.CSS_SELECTOR, 'div[class*="property-type-collpase_infoBlock"]')

# info_blocks[0].get_attribute('outerHTML')
# #property name
# name_ele = info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_key"]')
# #property value
# price_ele = info_blocks[0].find_element(By.CSS_SELECTOR, ':scope > div:nth-child(2) div[class*="property-type-collpase_value"]')

# print(driver.execute_script("return arguments[0].textContent;", name_ele))
# print(driver.execute_script("return arguments[0].textContent;", price_ele))

In [ ]:
# html_to_file(requests.get(page_url).text, 'city_page_html.html')

#### Get page links for Islamabad

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.zameen.com/new-projects/islamabad-3-1/')

In [15]:
city_project_urls = []

In [63]:
proj_tags = driver.find_elements(By.CSS_SELECTOR, "main > div > div > div > div > div > a")
urls = [ele.get_attribute('href') for ele in proj_tags]
city_project_urls.extend(urls)


In [64]:
city_project_urls

['https://www.zameen.com/new-projects/grand_orchard-2275/',
 'https://www.zameen.com/new-projects/airport_enclave_villas-2434/',
 'https://www.zameen.com/new-projects/360_the_residences-2241/',
 'https://www.zameen.com/new-projects/pearl_square_residency-2302/',
 'https://www.zameen.com/new-projects/rjs_lifestyle_residences-2024/',
 'https://www.zameen.com/new-projects/rjs_square-2269/',
 'https://www.zameen.com/new-projects/sparco_tower-2280/',
 'https://www.zameen.com/new-projects/jasmine_suites-2296/',
 'https://www.zameen.com/new-projects/river_hills_5-2226/',
 'https://www.zameen.com/new-projects/nova_city-2250/',
 'https://www.zameen.com/new-projects/hyde_park_one-2148/',
 'https://www.zameen.com/new-projects/grande_palladium-2090/',
 'https://www.zameen.com/new-projects/roman_grove-1940/',
 'https://www.zameen.com/new-projects/zameen_ace_mall-1623/',
 'https://www.zameen.com/new-projects/zameen_ace_homes-1694/',
 'https://www.zameen.com/new-projects/_safa_burj_mall_-476/',
 'htt

In [69]:
list(set(city_project_urls))

['https://www.zameen.com/new-projects/gulberg_twin_towers-1659/',
 'https://www.zameen.com/new-projects/kiyani_town_phase_3-1387/',
 'https://www.zameen.com/new-projects/pakland_medical_center-1177/',
 'https://www.zameen.com/new-projects/elite_arcade-1971/',
 'https://www.zameen.com/new-projects/mengal_heights-752/',
 'https://www.zameen.com/new-projects/mirador-640/',
 'https://www.zameen.com/new-projects/grand_orchard-2275/',
 'https://www.zameen.com/new-projects/al_cazar_heights-1046/',
 'https://www.zameen.com/new-projects/new_islamabad_garden_housing_scheme-775/',
 'https://www.zameen.com/new-projects/razia_terrace-788/',
 'https://www.zameen.com/new-projects/shahs_arcade-266/',
 'https://www.zameen.com/new-projects/tauheed_enclave-1431/',
 'https://www.zameen.com/new-projects/airport_enclave_villas-2434/',
 'https://www.zameen.com/new-projects/cbr_residencia-1421/',
 'https://www.zameen.com/new-projects/meher_apartments-941/',
 'https://www.zameen.com/new-projects/aries_tower_is